In [ ]:
#!pip install pymongo

In [2]:
import os

CONNECTION_STRING = "AZURE COSMOS DB MONGO vCORE connection string"
INDEX_NAME = "genaipoc-test-index"
NAMESPACE = "genaipoc_test_db.genaipoc_test_collection"
DB_NAME, COLLECTION_NAME = NAMESPACE.split(".")

In [1]:
#read .env file
from dotenv import load_dotenv
load_dotenv()

# os.getenv('OPENAI_API_KEY')
# os.getenv('OPENAI_API_VERSION')
# os.getenv('AZURE_OPENAI_ENDPOINT')
# os.environ[
#     "OPENAI_API_BASE"
# ] = "YOUR_OPEN_AI_ENDPOINT"  # https://example.openai.azure.com/
#os.getenv('OPENAI_API_KEY')
# os.environ[
#     "OPENAI_EMBEDDINGS_DEPLOYMENT"
# ] = "smart-agent-embedding-ada"  # the deployment name for the embedding model
#os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME")



True

In [4]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.azure_cosmos_db import (
    AzureCosmosDBVectorSearch,
    CosmosDBSimilarityType,
)

In [ ]:
SOURCE_FILE_NAME = "../../data/financebench/AMAZON_2019_10K.pdf"

loader = TextLoader(SOURCE_FILE_NAME)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [ ]:



# OpenAI Settings
model_deployment = os.getenv(
    "OPENAI_EMBEDDINGS_DEPLOYMENT", "smart-agent-embedding-ada"
)
model_name = os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME", "text-embedding-ada-002")


openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    deployment=model_deployment, model=model_name, chunk_size=1
)